In [1]:
import sys
import serial
import serial.tools.list_ports
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QMenuBar, QComboBox, QLabel, QAction, QFileDialog, QTextEdit
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore
import pandas as pd

class RealTimePlotApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

        self.plot_widget = pg.PlotWidget()
        self.plot_item = self.plot_widget.getPlotItem()
        self.plot_item.setTitle("Real-time Time Series Plot")
        self.plot_item.setLabel('left', 'Y-axis Label')
        self.plot_item.setLabel('bottom', 'X-axis Label')

        self.x_data = []
        self.y_data = []

        self.serial_port = None  # Store the serial port instance

        self.log_textedit = QTextEdit()
        self.log_textedit.setReadOnly(True)  # Make the log window read-only

        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()
        layout.addWidget(self.plot_widget)
        layout.addWidget(self.log_textedit)  # Add the log window to the layout
        self.central_widget.setLayout(layout)

        # Create a menu bar
        menubar = self.menuBar()

        # Control menu
        control_menu = menubar.addMenu('Control')

        # Program IC action
        program_ic_action = QAction('Program IC', self)
        program_ic_action.triggered.connect(self.program_ic)
        control_menu.addAction(program_ic_action)

        # Power Up IC action
        power_up_ic_action = QAction('Power Up IC', self)
        power_up_ic_action.triggered.connect(self.power_up_ic)
        control_menu.addAction(power_up_ic_action)

        # Serial Port menu
        self.serial_port_label = QLabel("Serial Port:")
        self.serial_port_combobox = QComboBox()
        layout.addWidget(self.serial_port_label)
        layout.addWidget(self.serial_port_combobox)  # Add the combobox to the central widget's layout
        self.populate_serial_ports()

        # Baud Rate menu
        self.baudrate_label = QLabel("Baud Rate:")
        self.baudrate_combobox = QComboBox()
        layout.addWidget(self.baudrate_label)
        layout.addWidget(self.baudrate_combobox)  # Add the combobox to the central widget's layout
        self.baudrate_combobox.addItems(['9600', '115200', '2000000'])  # Add your desired baud rates

        self.show()

    def populate_serial_ports(self):
        ports = [port.device for port in serial.tools.list_ports.comports()]
        self.serial_port_combobox.clear()
        self.serial_port_combobox.addItems(ports)

    def connect_serial(self):
        if self.serial_port and self.serial_port.is_open:
            # Close the serial port if already open
            self.serial_port.close()
            self.connect_button.setText("Connect")
        else:
            # Open the selected serial port
            port_name = self.serial_port_combobox.currentText()
            baud_rate = int(self.baudrate_combobox.currentText())

            try:
                self.serial_port = serial.Serial(port_name, baud_rate, timeout=1)
                self.connect_button.setText("Disconnect")
            except serial.SerialException as e:
                self.log_message(f"Error opening serial port: {e}")

    def program_ic(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Excel File", "", "Excel Files (*.xlsx);;All Files (*)", options=options)

        if file_name:
            df = pd.read_excel(file_name)
            self.log_message(f"Opened Excel file: {file_name}")
            print(df.head())  # Display the first few rows of the DataFrame

    def power_up_ic(self):
        if self.serial_port and self.serial_port.is_open:
            # Send a bytearray with values [36, 36, 49]
            data_to_send = bytearray([36, 36, 49])
            self.serial_port.write(data_to_send)
            self.log_message("Sent 'Power Up IC' command")
        else:
            self.log_message("Serial port not open. Cannot send 'Power Up IC' command.")

    def log_message(self, message):
        # Print log messages to the log window
        self.log_textedit.append(message)

    def update_plot(self):
        if self.serial_port and self.serial_port.is_open:
            # Read data from the serial port (modify as needed)
            data = self.serial_port.readline().decode('utf-8').strip()

            # Split data into x and y values (modify as needed)
            try:
                x, y = map(float, data.split(','))

                # Append data to the lists
                self.x_data.append(x)
                self.y_data.append(y)

                # Update the plot
                self.plot_item.plot(self.x_data, self.y_data, clear=True)
            except ValueError as e:
                self.log_message(f"Error parsing data: {e}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = RealTimePlotApp()
    sys.exit(app.exec_())


C:\Users\jahaveri\AppData\Local\Temp\ipykernel_30840\1303974210.py:9: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class RealTimePlotApp(QMainWindow):


SystemExit: 0

C:\Users\jahaveri\.conda\envs\biolidar\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
